<h3>Mita's Submission: Capstone Project Week 3</h3>

In [1]:
import pandas as pd
!conda install beautifulsoup4 #install beautifulsoup4
from bs4 import BeautifulSoup 
import requests # library to handle requests
import lxml # import html parser

print ('Libraries imported.')

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0

beautifulsoup4 100% |################################| Time: 0:00:00  41.89 MB/s
Libraries imported.


In [2]:
#define url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#get data from web page
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

#get table 
table_body=soup.find('tbody')



In [3]:

#initialize variable 
postcode = ""

#create dataframe
df = pd.DataFrame(columns=['PostalCode','Borough', 'Neighbourhood'])

#Go through each row of the table and pull out our data
for row in table_body.find_all("tr"):
    cells = row.find_all("td")
    if len(cells) > 0:
        postcode_previous = postcode
        postcode = cells[0].text.strip()
        borough = cells[1].text.strip()
        neighbourhood = cells[2].text.strip()
        #only continue if borough has a proper value (anything but 'not assigned')
        if borough != 'Not assigned':
            if postcode == postcode_previous: #compare current postalcode value to previous one
                #df.drop(df.tail(1).index,inplace=True) 
                df.drop(df.index[df['PostalCode'] == postcode], inplace = True) # drop last row since we wrote it before checking if postcode was the same as the previous one
                neighbourhood_concat = neighbourhood + ", " + neighbourhood_concat #create concatenated neighbourhood string
            else:
                if neighbourhood == 'Not assigned': #assign borough value to neighbourhood if it is not assigned
                    neighbourhood_concat = borough
                else:
                    neighbourhood_concat = neighbourhood #keep neighbourhood value if it exists
            #write row to dataframe once all variables are set
            df = df.append({'PostalCode': postcode, 'Borough': borough, 'Neighbourhood': neighbourhood_concat}, ignore_index=True)  
    

            
df.shape



(103, 3)

In [4]:
df

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [16]:
#read lat/long csv file into dataframe

latlong_df = pd.read_csv("https://cocl.us/Geospatial_data") 
latlong_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#rename PostalCode column in neighbourhoods dataframe to match that of lat long table
df.rename(columns={"PostalCode": "Postal Code"}, inplace = True)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park


In [22]:
#join latlong table to neighbourhood table
df_merged = pd.merge(df, latlong_df, how='inner', on = 'Postal Code')

In [23]:
df_merged.head(20)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<h2>Capstone Week 3 - Part 3
<h3>Explore and cluster the neighborhoods in Toronto. Work with only boroughs that contain the word Toronto.


In [29]:

#create new dataframe containing only the neighbourhoods with the name 'Toronto' in them
toronto_data = df_merged[df_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)

#check how many boroughs and neighbourhoods there are
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


In [24]:
#use geopy library to get lat and long values of Toronto

#install geopy library
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.50 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.56 MB/s
The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [30]:
#install folium library
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [31]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map using data from toronto_data dataframe
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto